In [1]:
import numpy as np

def simulate_energy_prices(S0, mu, sigma, T, dt, n_sim):
    n_steps = int(T / dt)
    prices = np.zeros((n_sim, n_steps))
    prices[:, 0] = S0

    for t in range(1, n_steps):
        Z = np.random.normal(0, 1, n_sim)  # Standard normal random variables
        prices[:, t] = prices[:, t-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
    
    return prices


In [53]:
class BatteryStorage:
    def __init__(self, max_capacity, min_capacity, max_charge_rate, max_discharge_rate, round_trip_efficiency):
        self.max_capacity = max_capacity
        self.min_capacity = min_capacity
        self.max_charge_rate = max_charge_rate
        self.max_discharge_rate = max_discharge_rate
        self.charge_efficiency = np.sqrt(round_trip_efficiency)  # Charge efficiency
        self.discharge_efficiency = np.sqrt(round_trip_efficiency)  # Discharge efficiency

    def can_charge(self, future_storage):
        return future_storage >= self.min_capacity

    def can_discharge(self, future_storage):
        return future_storage <= self.max_capacity


In [54]:
def bess_valuation(prices, battery, discount_rate, dt):
    n_sim, n_steps = prices.shape
    V = np.zeros((n_sim, n_steps))  # Storage value at each time step
    optimal_decision = np.zeros((n_sim, n_steps))  # Store the optimal decision
    state_of_charge = np.zeros((n_sim, n_steps))  # Track the current SoC for each path
    
    # Initial SoC (can be set to any initial level)
    initial_soc = 0  # Start with an empty battery for simplicity
    state_of_charge[:, 0] = initial_soc
    state_of_charge[:, -1] = initial_soc  # Ensure the battery is empty at the end of the simulation

    # Backward induction for each time step
    for t in range(n_steps - 2, -1, -1):
        for i in range(n_sim):
            current_price = prices[i, t]
            future_price = prices[i, t + 1]  # Price at the next time step
            future_storage = state_of_charge[i, t +1]  # Current SoC

            # No action (hold storage)
            hold_value = V[i, t + 1] * np.exp(-discount_rate * dt)

            # Charge the battery (if possible)
            if battery.can_charge(future_storage):
                energy_to_charge = min(battery.max_charge_rate * dt, future_storage - battery.min_capacity)
                charge_cost = current_price * energy_to_charge
                new_storage = future_storage + energy_to_charge * battery.charge_efficiency
                future_value_charge = V[i, t + 1] * np.exp(-discount_rate * dt)
                charge_value = -charge_cost + future_value_charge  # Cost to charge, plus future value
            else:
                charge_value = -np.inf  # Can't charge if storage is full

            # Discharge the battery (if possible)
            if battery.can_discharge(future_storage):
                energy_to_discharge = min(battery.max_discharge_rate * dt, battery.max_capacity - future_storage)
                discharge_revenue = future_price * energy_to_discharge
                new_storage = future_storage - energy_to_discharge / battery.discharge_efficiency
                future_value_discharge = V[i, t + 1] * np.exp(-discount_rate * dt)
                discharge_value = discharge_revenue + future_value_discharge  # Revenue from discharging, plus future value
            else:
                discharge_value = -np.inf  # Can't discharge if storage is empty

            # Choose the best option (charge, discharge, or hold)
            best_value = max(hold_value, charge_value, discharge_value)
            V[i, t] = best_value

            if best_value == charge_value:
                optimal_decision[i, t] = 1  # 1 = Charge
                state_of_charge[i, t] = future_storage - energy_to_charge * battery.charge_efficiency  # Update SoC after charging
            elif best_value == discharge_value:
                optimal_decision[i, t] = -1  # -1 = Discharge
                state_of_charge[i, t] = future_storage + energy_to_discharge / battery.discharge_efficiency  # Update SoC after discharging
            else:
                optimal_decision[i, t] = future_storage  # 0 = Hold (no action)
        

    return V, optimal_decision, state_of_charge


In [58]:
# Parameters
S0 = 100  # Initial electricity price (e.g., $/MWh)
mu = 0  # Risk-neutral drift (for simplicity)
sigma = 0.2  # Price volatility
T = 100  # 1 year
dt = 1  # Monthly time steps
n_sim = 1000  # Number of simulations
discount_rate = 0  # Discount rate

# BESS parameters
max_capacity = 200  # Maximum storage level in MWh
min_capacity = 0  # Minimum storage level (0 MWh)
max_charge_rate = 10  # Maximum charge rate in MW
max_discharge_rate = 10  # Maximum discharge rate in MW
round_trip_efficiency = 1  # 90% round-trip efficiency

# Simulate electricity prices
prices = simulate_energy_prices(S0, mu, sigma, T, dt, n_sim)

# Create battery storage object
battery = BatteryStorage(max_capacity, min_capacity, max_charge_rate, max_discharge_rate, round_trip_efficiency)

# Run Monte Carlo valuation
V, decisions, states_of_charge = bess_valuation(prices, battery, discount_rate, dt)

# Results
print("Simulated Electricity Prices (first few paths):", prices[:5, :])
print("BESS Valuation (first few paths):", V[:5, :])
print("Optimal Decisions (first few paths):", decisions[:5, :])
print("Optimal Storage Level (first few paths):", states_of_charge[:5, :])  # Total charge - discharge decisions


Simulated Electricity Prices (first few paths): [[1.00000000e+02 1.10188623e+02 1.07665721e+02 1.30445238e+02
  1.30364982e+02 1.48837990e+02 1.28146735e+02 1.04351131e+02
  9.12951822e+01 8.25072956e+01 8.43008027e+01 8.17731852e+01
  6.98738834e+01 6.92472077e+01 5.78235567e+01 3.74421107e+01
  3.50109829e+01 3.56809281e+01 3.83453190e+01 3.27428031e+01
  2.22029006e+01 1.77894789e+01 1.55768942e+01 2.11631558e+01
  2.10101714e+01 2.28727397e+01 2.52870005e+01 2.39240805e+01
  2.56919145e+01 2.59832952e+01 2.13785083e+01 2.11336467e+01
  1.65522749e+01 1.33273554e+01 1.10421516e+01 1.20442960e+01
  1.10826691e+01 9.80732587e+00 1.29474093e+01 1.27494524e+01
  1.69102446e+01 1.65765965e+01 1.18464248e+01 9.63841331e+00
  7.28635577e+00 9.38209587e+00 6.47777699e+00 5.86953486e+00
  5.64742579e+00 4.45225379e+00 3.42708970e+00 3.10055766e+00
  3.15349738e+00 2.95347626e+00 2.14446072e+00 1.84656779e+00
  1.67654617e+00 1.74847763e+00 1.66676037e+00 2.24606206e+00
  2.44820125e+00 2.511

In [18]:
import numpy as np

class BESS_LSMC:
    """Class for Battery Energy Storage System (BESS) valuation using Least-Squares Monte Carlo (LSMC).
    
    Parameters:
    ----------
    S0 : float : initial electricity price (e.g., $/MWh)
    T : float : time horizon (in year fractions)
    M : int : time steps (granularity of time grid)
    r : float : discount rate (e.g., 5%)
    sigma : float : volatility of electricity prices
    simulations : int : number of Monte Carlo paths
    bess_params : dict : parameters for the battery (max capacity, charge/discharge rates, round-trip efficiency)
    """

    def __init__(self, S0, T, M, r, sigma, simulations, bess_params):
        self.S0 = S0  # Initial electricity price
        self.T = T  # Time to maturity (e.g., 1 year)
        self.M = M  # Number of time steps
        self.r = r  # Risk-free rate
        self.sigma = sigma  # Volatility
        self.simulations = simulations  # Number of Monte Carlo simulations

        # Battery storage parameters
        self.max_capacity = bess_params['max_capacity']
        self.max_charge_rate = bess_params['max_charge_rate']
        self.max_discharge_rate = bess_params['max_discharge_rate']
        self.charge_efficiency = bess_params['charge_efficiency']
        self.discharge_efficiency = bess_params['discharge_efficiency']

        # Derived properties
        self.time_unit = T / float(M)
        self.discount = np.exp(-r * self.time_unit)

    def simulate_price_paths(self, seed=123):
        """Simulates the price paths using Geometric Brownian Motion (GBM)."""
        np.random.seed(seed)
        price_paths = np.zeros((self.M + 1, self.simulations))
        price_paths[0, :] = self.S0
        
        for t in range(1, self.M + 1):
            brownian = np.random.standard_normal(self.simulations // 2)
            brownian = np.concatenate((brownian, -brownian))
            price_paths[t, :] = price_paths[t - 1, :] * np.exp((self.r - 0.5 * self.sigma**2) * self.time_unit
                                                                + self.sigma * brownian * np.sqrt(self.time_unit))
        return price_paths

    def regression_basis(self, x):
        """Basis functions for regression (e.g., polynomial basis)."""
        return np.vstack([np.ones(len(x)), x, x**2, x**3, x**4, x**5]).T

    def calculate_bess_valuation(self, price_paths):
        """Calculates the BESS value using Least-Squares Monte Carlo."""
        n_steps, n_sim = price_paths.shape
        state_of_charge = np.zeros(n_sim)  # Initialize with an empty battery
        cashflows = np.zeros((n_steps, n_sim))  # Cashflows from charging/discharging

        # Final cashflows at the last timestep are zero since there's no future value at that point
        cashflows[-1, :] = 0

        # Backward induction
        for t in range(n_steps - 2, 0, -1):
            # Find paths where it makes sense to charge or discharge (in-the-money)
            charge_indices = state_of_charge < self.max_capacity
            discharge_indices = state_of_charge > 0

            # Cashflows for charging
            cashflow_if_charge = np.zeros(n_sim)
            energy_to_charge = np.minimum(self.max_charge_rate * self.time_unit, 
                                          self.max_capacity - state_of_charge[charge_indices])
            cashflow_if_charge[charge_indices] = -price_paths[t, charge_indices] * energy_to_charge * \
                                                 self.charge_efficiency

            # Cashflows for discharging
            cashflow_if_discharge = np.zeros(n_sim)
            energy_to_discharge = np.minimum(self.max_discharge_rate * self.time_unit, 
                                             state_of_charge[discharge_indices])
            cashflow_if_discharge[discharge_indices] = price_paths[t, discharge_indices] * energy_to_discharge * \
                                                      self.discharge_efficiency

            # Combine and discount future cashflows
            cashflow_if_hold = cashflows[t + 1, :] * self.discount

            # Regression for continuation value
            basis = self.regression_basis(price_paths[t, :])
            regression_coeffs = np.linalg.lstsq(basis, cashflow_if_hold, rcond=None)[0]
            continuation_value = np.dot(basis, regression_coeffs)

            # Decision: Charge, Discharge, or Hold
            for i in range(n_sim):
                if charge_indices[i]:
                    charge_value = cashflow_if_charge[i] + continuation_value[i]
                else:
                    charge_value = -np.inf

                if discharge_indices[i]:
                    discharge_value = cashflow_if_discharge[i] + continuation_value[i]
                else:
                    discharge_value = -np.inf

                hold_value = continuation_value[i]

                # Choose the maximum value (best decision)
                best_value = max(charge_value, discharge_value, hold_value)
                cashflows[t, i] = best_value

                # Update state of charge based on decision
                if best_value == charge_value:
                    state_of_charge[i] += self.max_charge_rate * self.time_unit * self.charge_efficiency
                elif best_value == discharge_value:
                    state_of_charge[i] -= self.max_discharge_rate * self.time_unit / self.discharge_efficiency

        # Discounted value at the first step
        return np.mean(cashflows[1, :]) * self.discount

# Example Usage

# Parameters
S0 = 100  # Initial electricity price ($/MWh)
T = 1  # Time horizon (1 year)
M = 12  # Monthly time steps
r = 0.05  # Discount rate (5%)
sigma = 0.2  # Volatility (20%)
simulations = 1000  # Number of Monte Carlo simulations
bess_params = {
    'max_capacity': 100,  # Maximum storage capacity (MWh)
    'max_charge_rate': 10,  # Maximum charge rate (MW)
    'max_discharge_rate': 10,  # Maximum discharge rate (MW)
    'charge_efficiency': 0.9,  # Charge efficiency (90%)
    'discharge_efficiency': 0.9  # Discharge efficiency (90%)
}

# Create BESS LSMC model
bess_model = BESS_LSMC(S0, T, M, r, sigma, simulations, bess_params)

# Simulate price paths
price_paths = bess_model.simulate_price_paths()

# Calculate BESS valuation using LSMC
bess_value = bess_model.calculate_bess_valuation(price_paths)

print(f"Estimated BESS Value: {bess_value:.2f}")


Estimated BESS Value: 0.00
